## Implementing the Byte-Pair encoding method (as opposed to single word encoding)


For example in context - LLama uses vocabulary size of 32K Tokens, GPT3 478 base characters and 10 000 vocabulary merges.

Depends on the structure of the language, bpe in English vs in Chinese would follow different rules - like a different take on spaces

Byte-level BPE: base vocabulary constructed not with unicode characters but bytes

### Tokenization

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2") #gpt3 and 2 have the same encoding

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
!pip3 install datasets

In [ ]:
from datasets import load_dataset

ds=load_dataset("tatsu-lab/alpaca")

In [ ]:
ds=ds.select_columns("instruction")
print(ds[0])
ds=[d['instruction'] for d in ds]

{'instruction': 'Give three tips for staying healthy.'}


In [ ]:
print(ds[:5])

['Give three tips for staying healthy.', 'What are the three primary colors?', 'Describe the structure of an atom.', 'How can we reduce air pollution?', 'Describe a time when you had to make a difficult decision.']


In [ ]:
from collections import defaultdict

word_freqs = defaultdict(int)

for text in ds:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)

defaultdict(<class 'int'>, {'Give': 932, 'Ġthree': 2135, 'Ġtips': 265, 'Ġfor': 7814, 'Ġstaying': 21, 'Ġhealthy': 236, '.': 36939, 'What': 2317, 'Ġare': 1666, 'Ġthe': 42202, 'Ġprimary': 103, 'Ġcolors': 82, '?': 4081, 'Describe': 2972, 'Ġstructure': 259, 'Ġof': 20274, 'Ġan': 6316, 'Ġatom': 20, 'How': 949, 'Ġcan': 1489, 'Ġwe': 112, 'Ġreduce': 390, 'Ġair': 122, 'Ġpollution': 138, 'Ġa': 35893, 'Ġtime': 369, 'Ġwhen': 402, 'Ġyou': 1497, 'Ġhad': 148, 'Ġto': 12315, 'Ġmake': 1150, 'Ġdifficult': 74, 'Ġdecision': 106, 'Identify': 1511, 'Ġodd': 54, 'Ġone': 1370, 'Ġout': 383, 'Explain': 2018, 'Ġwhy': 1028, 'Ġfollowing': 7398, 'Ġfraction': 28, 'Ġis': 4656, 'Ġequivalent': 59, 'Ġ1': 385, '/': 268, '4': 103, 'Write': 2761, 'Ġshort': 554, 'Ġstory': 1628, 'Ġin': 9013, 'Ġthird': 97, 'Ġperson': 660, 'Ġnarration': 3, 'Ġabout': 2952, 'Ġprotagonist': 74, 'Ġwho': 540, 'Ġhas': 531, 'Ġimportant': 530, 'Ġcareer': 64, 'Render': 5, 'Ġ3': 1256, 'D': 58, 'Ġmodel': 370, 'Ġhouse': 96, 'Evaluate': 171, 'Ġthis': 2662, 'Ġs

In [ ]:
chars = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in chars:
            chars.append(letter)
chars.sort()

In [ ]:
print(chars)

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '¡', '¢', '£', '¥', '¦', '§', '¨', '©', 'ª', '°', '±', '³', '´', '·', '¸', '¹', '¼', '¿', 'Â', 'Ã', 'Ï', 'â', 'ê', 'ë', 'ì', 'ï', 'ð', 'Ċ', 'Ġ', 'Ģ', 'ģ', 'Ĩ', 'Ī', 'ĭ', 'ı', 'Ĵ', 'ĵ', 'Ķ', 'Ĺ', 'ĺ', 'Ļ', 'ļ', 'ľ', 'Ŀ', 'ŀ', 'Ł', 'ł', 'Ń']


In [ ]:
# add special tokens
chars.append("<|endoftext|>")

Split each word into characters

In [ ]:
wordsplits = {word: [c for c in word] for word in word_freqs.keys()}

In [ ]:
print(wordsplits)

{'Give': ['G', 'i', 'v', 'e'], 'Ġthree': ['Ġ', 't', 'h', 'r', 'e', 'e'], 'Ġtips': ['Ġ', 't', 'i', 'p', 's'], 'Ġfor': ['Ġ', 'f', 'o', 'r'], 'Ġstaying': ['Ġ', 's', 't', 'a', 'y', 'i', 'n', 'g'], 'Ġhealthy': ['Ġ', 'h', 'e', 'a', 'l', 't', 'h', 'y'], '.': ['.'], 'What': ['W', 'h', 'a', 't'], 'Ġare': ['Ġ', 'a', 'r', 'e'], 'Ġthe': ['Ġ', 't', 'h', 'e'], 'Ġprimary': ['Ġ', 'p', 'r', 'i', 'm', 'a', 'r', 'y'], 'Ġcolors': ['Ġ', 'c', 'o', 'l', 'o', 'r', 's'], '?': ['?'], 'Describe': ['D', 'e', 's', 'c', 'r', 'i', 'b', 'e'], 'Ġstructure': ['Ġ', 's', 't', 'r', 'u', 'c', 't', 'u', 'r', 'e'], 'Ġof': ['Ġ', 'o', 'f'], 'Ġan': ['Ġ', 'a', 'n'], 'Ġatom': ['Ġ', 'a', 't', 'o', 'm'], 'How': ['H', 'o', 'w'], 'Ġcan': ['Ġ', 'c', 'a', 'n'], 'Ġwe': ['Ġ', 'w', 'e'], 'Ġreduce': ['Ġ', 'r', 'e', 'd', 'u', 'c', 'e'], 'Ġair': ['Ġ', 'a', 'i', 'r'], 'Ġpollution': ['Ġ', 'p', 'o', 'l', 'l', 'u', 't', 'i', 'o', 'n'], 'Ġa': ['Ġ', 'a'], 'Ġtime': ['Ġ', 't', 'i', 'm', 'e'], 'Ġwhen': ['Ġ', 'w', 'h', 'e', 'n'], 'Ġyou': ['Ġ', 'y', 'o

calculate the frequency of a pair in given set

In [ ]:
def frequencepairs(splits):
    pair_freqs = defaultdict(int)

    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

Let's see an example with our character split

In [ ]:
freqpair = frequencepairs(wordsplits)

for i, key in enumerate(freqpair.keys()):
    print(f"{key}: {freqpair[key]}")
    if i >= 5:
        break

('G', 'i'): 3189
('i', 'v'): 15517
('v', 'e'): 22147
('Ġ', 't'): 82754
('t', 'h'): 64324
('h', 'r'): 3536


In [ ]:
def bestpair(freqspairs):
  maxi=0
  best=""

  for pair, freq in freqspairs.items():
      if maxi < freq:
          best = pair
          maxi = freq

  return(best, maxi)

In [ ]:
print(bestpair(freqpair))

(('Ġ', 't'), 82754)


In [ ]:
merges ={}

create a function to update the split of each word that contains this combination

In [ ]:
def splitmerge(a, b, splits):

    for word in splits:
        decomposed = splits[word]
        # decomposed is a list of symbols forming the word
        if len(decomposed) == 1:
            continue

        i = 0
        # loop over all possible pairs in a the split of that word
        while i < len(decomposed) - 1:
            if decomposed[i] == a and decomposed[i + 1] == b:
                # replace two individual symbol by merged symbol
                decomposed = decomposed[:i] + [a + b] + decomposed[i + 2 :]
            else:
                i += 1
        splits[word] = decomposed


    return splits

In [34]:
vocab=chars.copy()
vocab_size = 1000
# desired size of known vocabulary

while len(vocab) < vocab_size:
  print(len(vocab))
  freqpair = frequencepairs(wordsplits)
  best, maxfreq=bestpair(freqpair)

  wordsplits = splitmerge(*best, wordsplits)

  merges[best] = best[0] + best[1]
  vocab.append(best[0] + best[1])

143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392


In [36]:
print(vocab[900:])

['Formulate', 'Ġaffect', 'Ġunderstand', 'eline', 'Ġteach', 'Ġment', 'to', 'face', 'Ġvi', 'aphrase', 'Ġbegin', 'anish', 'QL', 'Ġlearn', 'arth', 'Ġsummer', 'Ġchall', 'Ġvisual', 'Ġgener', 'Ġrate', 'ĠSpanish', 'Ġqual', 'Ġsingle', 'Ġplace', 'Ġtool', 'Ġant', 'Ġchanging', 'Ass', 'Ġsecond', 'Ġpres', 'Reword', 'ves', 'Ġadjective', 'Ġbeen', 'sert', '19', 'Ġdetail', 'Ġpublic', 'Ġland', 'Ġassociated', 'air', 'Ġskill', 'ough', 'Ġ=', 'Ġtra', 'Ġtag', 'Ġformat', 'Ġtakes', 'ets', 'sent', 'Ġyears', 'Ġsupport', 'Ġsummarize', 'Ġrain', 'Ġpunctuation', 'lock', 'Ġly', 'Ġconsider', 'Ġbreak', 'ely', 'Ġris', 'Ġconveys', 'Ġpassword', 'Paraphrase', 'Ġsurve', 'Ġlogical', 'ply', 'rib', 'Ġcommunication', 'ĠHTML', 'Ġright', 'Ġexerc', 'Ġcare', 'Ġdid', 'Ġx', 'Ġmax', 'ume', 'Ġfound', 'cessary', 'tual', 'Ġnouns', 'Ġmist', 'Propose', 'Ġpract', 'Ġadverb', 'Ġnames', 'Ġlow', 'Ġpand', 'Ġphysical', 'onym', 'Ġmot', 'sentence', 'ty', 'Ġ$', 'Ġtimeline', 'Ġsaf', 'St', 'Ġ12', 'Complete', 'atest']
